#3. Fine-Tuning: <br>
##(a) Choose one of the pre-trained summarization models from the previous part: google-t5/t5-small <br>

In [ ]:
pip install transformers datasets evaluate accelerate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=5798b04217eca6c49e9ac7cb6131714b2336cfad8f80ecbc2c46473ba66e846f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import accelerate
import transformers
import numpy as np
import pandas as pd
from transformers import pipeline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from datasets import load_dataset_builder
from datasets import load_dataset
import seaborn as sns

In [ ]:
dataset = load_dataset("samsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
train = load_dataset("samsum", split="train")
test = load_dataset("samsum", split="test")
val = load_dataset("samsum", split="validation")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM

##(b) Preprocess the dataset to fit it to the input format required by the chosen model, which may include tokenizing the dialogues and their summaries.

In [ ]:
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True) # Tokenize in batches?

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True) # Tokenize in batches?

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

##(c) Train the model on the SAMSum corpus, monitoring its performance on the validation set, and adjust its hyperparameters as needed.

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
#from transformers import Seq2SeqTrainer
from transformers import Trainer

In [ ]:
#from transformers import Seq2SeqTrainingArguments
from transformers import TrainingArguments
import evaluate


In [ ]:
rouge_metric = evaluate.load("rouge")

In [ ]:
import torch

In [ ]:
def compute_metrics(eval_pred):
    #print(eval_pred)
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
batch_size = 1
num_train_epochs = 3

args = TrainingArguments(
    output_dir="/content/drive/MyDrive/LLMPA3-4",
    evaluation_strategy="no",  # Disable evaluation during training
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    gradient_accumulation_steps=16,
    save_steps=100,
    fp16=True,
    eval_steps=None,  # Ensure no evaluation is performed at any specific steps
    # push_to_hub=True,
    # save_strategy='epoch'
)


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=None, # no evaluation
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# preds_output = trainer.predict(tokenized_datasets["validation"])
# preds_output.metrics

In [ ]:
trainer.train()

Step,Training Loss
500,2.345800
1000,2.146700
1500,2.069700
2000,2.064800
2500,2.049000


Checkpoint destination directory /content/drive/MyDrive/LLMPA3-4/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=2760, training_loss=2.1237710703974186, metrics={'train_runtime': 2794.3976, 'train_samples_per_second': 15.816, 'train_steps_per_second': 0.988, 'total_flos': 1761326309081088.0, 'train_loss': 2.1237710703974186, 'epoch': 3.0})

##(d) Employ appropriate metrics for evaluating summarization quality, such as ROUGE scores, which compare the generated summaries against the reference summaries.

### Evaluating model on validation dataset

In [1]:
pip install transformers datasets evaluate accelerate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=58613901f62567c740ec9ce193f854c41d1b23c50cfb0c915a8feba0a61841cc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
from tqdm import tqdm
import accelerate
import transformers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import pipeline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from datasets import load_dataset_builder
from datasets import load_dataset
import seaborn as sns
import torch
import evaluate
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset = load_dataset("samsum")
train = load_dataset("samsum", split="train")
test = load_dataset("samsum", split="test")
val = load_dataset("samsum", split="validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [7]:
rouge_metric = evaluate.load("rouge")

In [5]:
batch_size = 8
model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/LLMPA3-4/checkpoint-2700')
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/LLMPA3-4/checkpoint-2700")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
# Assuming test is your dataset
model.to(device)
for i in tqdm(range(0, len(val), batch_size)):
    batch = val[i:i + batch_size]
    dialogues = batch['dialogue']
    references = batch['summary']

    # Tokenize the dialogues
    inputs = tokenizer(dialogues, padding=True, truncation=True, return_tensors="pt").to(device)

    # Generate summaries from the model
    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,  # Adjust max length according to your requirements
        num_beams=4,     # Adjust num_beams for beam search
        early_stopping=True,
        decoder_start_token_id=tokenizer.pad_token_id  # Avoid generating from padding tokens
    )

    # Move outputs to CPU for further processing
    outputs = outputs.cpu()

    # Decode the generated summaries
    generated_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Add batch to ROUGE metric
    rouge_metric.add_batch(predictions=generated_summaries, references=references)

# Compute ROUGE scores
rouge_metric.compute()


100%|██████████| 103/103 [01:36<00:00,  1.07it/s]


{'rouge1': 0.3679927732207304,
 'rouge2': 0.15667185596054012,
 'rougeL': 0.30151443898439867,
 'rougeLsum': 0.3011121163338637}